<a href="https://colab.research.google.com/github/MehmetPekerr/reid/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Setup Project, Environment & Data

# ========================================================= PROJECT SETUP =========================================================
# Clone Repo
!git clone --quiet https://github.com/SurajDonthi/MTMCT-Person-Re-Identification.git
%cd ./MTMCT-Person-Re-Identification/

# Install gdown for reliable Google Drive downloads
!pip install --quiet gdown

# Modify requirements.txt to remove strict version constraints for torchtext and pytorch-lightning
# This allows pip to install compatible versions with the current environment
!sed -i 's/torchtext==0.8.0/torchtext/' requirements.txt
!sed -i 's/pytorch-lightning==1.2.1/pytorch-lightning/' requirements.txt

!pip3 install --quiet -r requirements.txt

# ++++++++++++++++++++++++ D A T A   D O W N L O A D ++++++++++++++++++++++++

# Download Market-1501 Dataset using gdown for reliability
!gdown --id 1ozb7D2cvc-TKVlLVCdYNWmTgZzEvNT4R -O ../market\ 1501.zip

# Extract Market Dataset
!unzip -qq ../market\ 1501.zip -d ./data/raw/
# Data Cleaning - Fix file names
!find ./data/raw/Market-1501-v15.09.15 -name "*.jpg.jpg" -exec rename -v 's/\.jpg.jpg$/\.jpg/i' {} \;

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# =================================================================================================================================


/content/MTMCT-Person-Re-Identification
ERROR: Could not find a version that satisfies the requirement torchtext==0.8.0 (from versions: 0.1.1, 0.2.0, 0.2.1, 0.2.3, 0.3.1, 0.4.0, 0.5.0, 0.6.0, 0.16.2, 0.17.2, 0.18.0)
ERROR: No matching distribution found for torchtext==0.8.0
--2026-01-22 11:50:33--  https://docs.google.com/uc?export=download&confirm=&id=1ozb7D2cvc-TKVlLVCdYNWmTgZzEvNT4R
Resolving docs.google.com (docs.google.com)... 74.125.24.139, 74.125.24.100, 74.125.24.113, ...
Connecting to docs.google.com (docs.google.com)|74.125.24.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1ozb7D2cvc-TKVlLVCdYNWmTgZzEvNT4R&export=download [following]
--2026-01-22 11:50:34--  https://drive.usercontent.google.com/download?id=1ozb7D2cvc-TKVlLVCdYNWmTgZzEvNT4R&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.130.132, 2404:6800:4003:c01::84
Connecting to drive.use

In [2]:
import os
from argparse import Namespace

from pathlib import Path
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
# from pytorch_lightning.profiler import AdvancedProfiler
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger
from pytorch_lightning.loggers.test_tube import TestTubeLogger

from mtmct_reid.data import ReIDDataModule
from mtmct_reid.engine import ST_ReID
from mtmct_reid.utils import save_args

import warnings

warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'pytorch_lightning'

# Training

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/

In [ ]:
args = dict(
    description = "Training and Predicting on Market-1501 Dataset",
    data_dir = "data/raw/Market-1501-v15.09.15",
    dataset = "market",
    log_path = "logs/",

    learning_rate = 0.1,
    criterion = "cross_entropy",
    rerank = False,
    # random_erasing = True,
    # color_jitter = True,

    gpus = 1,
    max_epochs = 60,
    precision = 16,
    train_batchsize = 64,
    val_batchsize = 64,
    test_batchsize = 64,
    limit_val_batches = 1,
    profiler = True,
    progress_bar_refresh_rate = 20
)

args = Namespace(**args)

In [ ]:
tt_logger = TestTubeLogger(save_dir=args.log_path, name="",
                            description=args.description,
                            )

log_dir = Path(tt_logger.save_dir) / f"version_{tt_logger.version}"

checkpoint_dir = log_dir / "checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)
chkpt_callback = ModelCheckpoint(checkpoint_dir,
                                    monitor='Loss/val_loss/dataloader_idx_2',   # Pytorch-lightning appends extra path
                                    save_last=True,
                                    mode='min',
                                    save_top_k=10,
                                    )

data_module = ReIDDataModule.from_argparse_args(args)

model = ST_ReID(data_module.num_classes, learning_rate=args.learning_rate,
                criterion=args.criterion, rerank=args.rerank)

save_args(args, log_dir)

trainer = Trainer.from_argparse_args(args, logger=[tt_logger],
                                        checkpoint_callback=chkpt_callback,
                                        profiler=True)  # AdvancedProfiler()

#
trainer.fit(model, data_module)

# Testing/Evaluation

Use the model to predict matches from other cameras.

In [ ]:
# Testing the loss & accuracy of the model
trainer.test(model)

# Save the results

In [ ]:
!zip -r /content/experiment_results.zip /content/MTMCT-Person-Re-Identification/logs
from google.colab import files
files.download("/content/experiment_results.zip")